In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2


In [ ]:
import os
from pathlib import Path
import random

import matplotlib.pyplot as plt
import pickle
import torch
import numpy as np

# from experiment_setup import get_model, set_random, build_estimator
from alpaca.model.ensemble import MLPEnsemble
from alpaca.model.mlp import MLP
from alpaca.dataloader.builder import build_dataset
from experiments.utils.data import scale, multiple_kfold

plt.rcParams['figure.facecolor'] = 'white'

In [ ]:
SEED = 10
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

torch.cuda.set_device(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False



In [ ]:
config = {
    'nn_runs': 100,
    'runs': 2,
    'max_runs': 20,
    # Our train config
   'layers': [8, 128, 128, 64, 1],
    'epochs': 10_000,
    'validation_step': 100,

    'patience': 10,
    'dropout_rate': 0.5,
    'dropout_uq': 0.5,
    'batch_size': 256,
    'l2_reg': 1e-5,

    'optimizer': {'type': 'Adam', 'lr': 0.01, 'weight_decay':1e-5},
    'n_split_runs': 3,
    'n_model_runs': 3,
    'n_ens': 5,
    'n_ue_runs': 5,
    'k_folds': 10,
    'verbose': False,
}


In [ ]:
train_opts = ['patience', 'dropout_rate', 'epochs', 'batch_size', 'validation_step', 'verbose']
config['train_opts'] = {k: config[k] for k in config if k in train_opts}


In [ ]:
# datasets = [
#     'boston_housing', 'concrete', 'energy_efficiency',
#     'kin8nm', 'naval_propulsion', 'ccpp', 'red_wine',
#     'yacht_hydrodynamics'
# ]

In [ ]:
dataset_name = 'kin8nm'
dataset = build_dataset(dataset_name, val_split=0.2)
x_train, y_train = dataset.dataset('train')
x_val, y_val = dataset.dataset('val')

x_train, x_val, x_scaler = scale(x_train, x_val)
y_train, y_val, y_scaler = scale(y_train, y_val)

print(dataset_name, x_train.shape)
config['layers'][0] = x_train.shape[-1]


In [ ]:
model = MLP(config['layers'])


In [ ]:
model.fit((x_train, y_train), (x_val, y_val))


In [ ]:
x_val_tensor = torch.tensor(x_val)
predictions = model(x_val_tensor.cuda()).cpu().detach().numpy()


In [ ]:
unscale = lambda y : y_scaler.inverse_transform(y)

scaled_errors = unscale(predictions) - unscale(y_val)
rmse_single = np.sqrt(np.mean(np.square(scaled_errors)))

In [ ]:
train_opts = ['patience', 'dropout_rate', 'epochs', 'batch_size', 'validation_step', 'verbose']
config['train_opts'] = {k: config[k] for k in config if k in train_opts}

In [ ]:

train_opts = config['train_opts'].copy()
ensemble = MLPEnsemble(config['layers'],
                    n_models=config['n_ens'],
                    reduction='mean')
ensemble.fit((x_train, y_train),
          (x_val, y_val),
          **train_opts)



In [ ]:
predictions = ensemble(x_val_tensor.cuda()).cpu().detach().numpy()
scaled_errors = unscale(predictions) - unscale(y_val)
rmse_ensemble = np.sqrt(np.mean(np.square(scaled_errors)))


In [ ]:
print(dataset_name)
print('Single', rmse_single)
print('Ensemble', rmse_ensemble)

